In [1]:
import pandas as pd
import os
import researchpy as rp
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error

In [2]:
os.chdir('D:/ML/genpact_problem-statement')
os.getcwd()

'D:\\ML\\genpact_problem-statement'

In [3]:
training_data=pd.read_csv('./training.csv')
test_data=pd.read_csv('./test.csv')

In [4]:
training_data.head()

,Unnamed: 0,Kilometeres,Fuel_Type,Doors,Automatic,HorsePower,MetallicCol,CC,Wt,SellingPrice,Age
0,1,46986,Diesel,3,0,90,1,2000,1165,13500,23
1,7,94612,Diesel,3,0,90,1,2000,1245,16900,27
2,9,19700,Petrol,3,0,192,0,1800,1185,21500,27
3,12,43610,Petrol,3,0,192,0,1800,1185,19950,22
4,15,34131,Petrol,3,0,192,1,1800,1185,22500,32


In [5]:
test_data.head()

,Unnamed: 0,Kilometeres,Fuel_Type,Doors,Automatic,HorsePower,MetallicCol,CC,Wt,SellingPrice,Age
0,389,140700,Diesel,5,0,69,1,1900,1110,7750,48
1,441,77270,Petrol,4,0,110,1,1600,1035,11950,54
2,579,33230,Petrol,3,0,110,0,1600,1050,11500,55
3,322,36269,Petrol,5,0,110,1,1600,1075,10750,36
4,1303,70932,Petrol,3,0,110,1,1600,1050,7450,77


In [6]:
#removing first column from training and test data both
training_data.drop('Unnamed: 0',axis=1,inplace=True)
test_data.drop('Unnamed: 0',axis=1,inplace=True)

In [7]:
"Training Null",training_data.isnull().sum(),"TEST Null",test_data.isnull().sum()

('Training Null', Kilometeres     0
 Fuel_Type       0
 Doors           0
 Automatic       0
 HorsePower      0
 MetallicCol     0
 CC              0
 Wt              0
 SellingPrice    0
 Age             0
 dtype: int64, 'TEST Null', Kilometeres     0
 Fuel_Type       0
 Doors           0
 Automatic       0
 HorsePower      0
 MetallicCol     0
 CC              0
 Wt              0
 SellingPrice    0
 Age             0
 dtype: int64)

In [8]:
#checking data types of features and labels
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
Kilometeres     958 non-null int64
Fuel_Type       958 non-null object
Doors           958 non-null int64
Automatic       958 non-null int64
HorsePower      958 non-null int64
MetallicCol     958 non-null int64
CC              958 non-null int64
Wt              958 non-null int64
SellingPrice    958 non-null int64
Age             958 non-null int64
dtypes: int64(9), object(1)
memory usage: 74.9+ KB


In [9]:
training_data.head()

,Kilometeres,Fuel_Type,Doors,Automatic,HorsePower,MetallicCol,CC,Wt,SellingPrice,Age
0,46986,Diesel,3,0,90,1,2000,1165,13500,23
1,94612,Diesel,3,0,90,1,2000,1245,16900,27
2,19700,Petrol,3,0,192,0,1800,1185,21500,27
3,43610,Petrol,3,0,192,0,1800,1185,19950,22
4,34131,Petrol,3,0,192,1,1800,1185,22500,32


In [10]:
rp.summary_cat(training_data['Fuel_Type'])

,Variable,Outcome,Count,Percent
0,Fuel_Type,Petrol,846,88.31
1,,Diesel,101,10.54
2,,CNG,11,1.15


In [11]:
rp.summary_cont(training_data[['Kilometeres','Doors','Automatic','HorsePower','MetallicCol','CC','Wt','Age','SellingPrice']])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,Kilometeres,958.0,69067.598121,37082.673742,1198.086806,66716.417542,71418.778700
1,Doors,958.0,4.009395,0.955059,0.030857,3.948840,4.069949
2,Automatic,958.0,0.056367,0.230750,0.007455,0.041737,0.070998
3,HorsePower,958.0,101.549061,14.556315,0.470293,100.626136,102.471985
4,MetallicCol,958.0,0.669102,0.470782,0.015210,0.639253,0.698952
5,CC,958.0,1566.651357,184.850459,5.972247,1554.931145,1578.371569
6,Wt,958.0,1072.701461,55.235835,1.784589,1069.199302,1076.203620
7,Age,958.0,56.463466,18.332019,0.592281,55.301147,57.625784
8,SellingPrice,958.0,10666.893528,3633.781940,117.402165,10436.498126,10897.288930


In [12]:
training_data.describe()

,Kilometeres,Doors,Automatic,HorsePower,MetallicCol,CC,Wt,SellingPrice,Age
count,958.000000,958.000000,958.000000,958.000000,958.000000,958.000000,958.000000,958.000000,958.000000
mean,69067.598121,4.009395,0.056367,101.549061,0.669102,1566.651357,1072.701461,10666.893528,56.463466
std,37082.673742,0.955059,0.230750,14.556315,0.470782,184.850459,55.235835,3633.781940,18.332019
min,1.000000,2.000000,0.000000,69.000000,0.000000,1300.000000,1000.000000,4400.000000,1.000000
25%,43472.000000,3.000000,0.000000,90.000000,0.000000,1400.000000,1040.000000,8450.000000,44.000000
50%,63547.000000,4.000000,0.000000,110.000000,1.000000,1600.000000,1065.000000,9900.000000,61.000000
75%,87580.000000,5.000000,0.000000,110.000000,1.000000,1600.000000,1085.000000,11750.000000,70.000000
max,243000.000000,5.000000,1.000000,192.000000,1.000000,2000.000000,1615.000000,32500.000000,80.000000


In [13]:
#kilometers has outliers
training_data.head()

,Kilometeres,Fuel_Type,Doors,Automatic,HorsePower,MetallicCol,CC,Wt,SellingPrice,Age
0,46986,Diesel,3,0,90,1,2000,1165,13500,23
1,94612,Diesel,3,0,90,1,2000,1245,16900,27
2,19700,Petrol,3,0,192,0,1800,1185,21500,27
3,43610,Petrol,3,0,192,0,1800,1185,19950,22
4,34131,Petrol,3,0,192,1,1800,1185,22500,32


In [14]:
training_data=pd.get_dummies(training_data,drop_first=True)

In [15]:
test_data=pd.get_dummies(test_data,drop_first=True)

In [16]:
training_data.shape,test_data.shape

((958, 11), (478, 11))

In [17]:
#Checking for correlation bw independent variables through corr matrix
training_data.corr()

,Kilometeres,Doors,Automatic,HorsePower,MetallicCol,CC,Wt,SellingPrice,Age,Fuel_Type_Diesel,Fuel_Type_Petrol
Kilometeres,1.000000,-0.037574,-0.097628,-0.329155,-0.077100,0.309269,-0.035067,-0.554376,0.484124,0.403781,-0.442045
Doors,-0.037574,1.000000,-0.016630,0.116733,0.090585,0.111654,0.282493,0.180088,-0.129402,0.003743,-0.013436
Automatic,-0.097628,-0.016630,1.000000,0.037441,-0.049360,-0.044713,0.071991,0.055216,0.003205,-0.083904,0.074842
HorsePower,-0.329155,0.116733,0.037441,1.000000,0.067252,0.042837,0.074950,0.291650,-0.138730,-0.544692,0.499847
MetallicCol,-0.077100,0.090585,-0.049360,0.067252,1.000000,0.055372,0.060330,0.083892,-0.087669,0.003039,-0.014226
CC,0.309269,0.111654,-0.044713,0.042837,0.055372,1.000000,0.616939,0.149342,-0.107144,0.760421,-0.733244
Wt,-0.035067,0.282493,0.071991,0.074950,0.060330,0.616939,1.000000,0.576559,-0.460229,0.544393,-0.537930
SellingPrice,-0.554376,0.180088,0.055216,0.291650,0.083892,0.149342,0.576559,1.000000,-0.870502,0.056058,-0.034852
Age,0.484124,-0.129402,0.003205,-0.138730,-0.087669,-0.107144,-0.460229,-0.870502,1.000000,-0.089566,0.085268
Fuel_Type_Diesel,0.403781,0.003743,-0.083904,-0.544692,0.003039,0.760421,0.544393,0.056058,-0.089566,1.000000,-0.943510


In [18]:
x_train=training_data.drop('SellingPrice',1)
y_train=training_data['SellingPrice']

In [19]:
vif=pd.DataFrame()
vif['VIF']=[variance_inflation_factor(x_train.values,i) for i in range(x_train.shape[1])]

In [20]:
vif['Features']=x_train.columns
vif

,VIF,Features
0,8.596210,Kilometeres
1,20.909482,Doors
2,1.100458,Automatic
3,266.579255,HorsePower
4,3.096578,MetallicCol
5,572.618382,CC
6,275.920261,Wt
7,17.093976,Age
8,23.726385,Fuel_Type_Diesel
9,71.522250,Fuel_Type_Petrol


In [21]:
x_test=test_data.drop('SellingPrice',1)
y_test=test_data['SellingPrice']

In [22]:
scaler=StandardScaler()

In [23]:
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  


In [24]:
model_LR=LinearRegression()
model_LR.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [25]:
y_pred_test_LR=model_LR.predict(x_test)

In [26]:
#R square of model
model_LR.score(x_test,y_test)

0.8854254723903674

In [27]:
rmse_LR=np.sqrt(mean_squared_error(y_test,y_pred_test_LR))
rmse_LR

1221.8998341534184

In [28]:
residual_LR = y_pred_test_LR - y_test

In [29]:
print('Mean Absolute Error:', mean_absolute_error(y_test,y_pred_test_LR), 'degrees.')
mape_LR = np.mean(100 * (abs(residual_LR) / y_test))
accuracy_LR = 100 - mape_LR
print("Mean Absolute percentage error",mape_LR)
print("Accuracy of model",accuracy_LR)

Mean Absolute Error: 935.1640917376262 degrees.
Mean Absolute percentage error 9.377208847697503
Accuracy of model 90.6227911523025


In [30]:
model_RF=RandomForestRegressor(n_estimators=1000,random_state = 10)
model_RF.fit(x_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
           oob_score=False, random_state=10, verbose=0, warm_start=False)

In [31]:
y_pred_test_RF=model_RF.predict(x_test)

In [32]:
#R square of model
model_RF.score(x_test,y_test)

0.9120333468649348

In [33]:
rmse_RF=np.sqrt(mean_squared_error(y_test,y_pred_test_RF))
rmse_RF

1070.65759601623

In [34]:
residual_RF = y_pred_test_RF - y_test

In [35]:
print('Mean Absolute Error:', mean_absolute_error(y_test,y_pred_test_RF), 'degrees.')
mape_RF = np.mean(100 * (abs(residual_RF) / y_test))
accuracy_RF = 100 - mape_RF
print("Mean Absolute percentage error",mape_RF)
print("Accuracy of model",accuracy_RF)

Mean Absolute Error: 825.4345231121737 degrees.
Mean Absolute percentage error 8.39357827104168
Accuracy of model 91.60642172895832


In [36]:
model_GB=GradientBoostingRegressor()
model_GB.fit(x_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [37]:
y_pred_test_GB=model_GB.predict(x_test)

In [38]:
model_GB.score(x_test,y_test)

0.9107127559621744

In [39]:
rmse_GB=np.sqrt(mean_squared_error(y_test,y_pred_test_GB))
rmse_GB

1078.6642302409043

In [40]:
residual_GB = y_pred_test_GB - y_test

In [41]:
print('Mean Absolute Error:', mean_absolute_error(y_test,y_pred_test_GB), 'degrees.')
mape_GB = np.mean(100 * (abs(residual_GB) / y_test))
accuracy_GB = 100 - mape_GB
print("Mean Absolute percentage error",mape_GB)
print("Accuracy of model",accuracy_GB)

Mean Absolute Error: 816.026564847514 degrees.
Mean Absolute percentage error 8.234680813375546
Accuracy of model 91.76531918662445


In [42]:
predicted = pd.DataFrame(columns=["Actual","Predicted_LR","Predicted_RF","Predicted_GB"])

In [43]:
predicted["Actual"]=y_test
predicted["Predicted_LR"]=y_pred_test_LR
predicted["Predicted_RF"]=y_pred_test_RF
predicted["Predicted_GB"]=y_pred_test_GB

In [44]:
predicted

,Actual,Predicted_LR,Predicted_RF,Predicted_GB
0,7750,10108.357427,8551.835000,8005.971989
1,11950,10291.594803,11092.460000,10508.384676
2,11500,11152.488587,10869.805000,10675.118015
3,10750,13962.732974,13306.730000,12726.301174
4,7450,7734.938481,8135.875000,8031.437124
5,9940,11908.644225,13059.160000,11611.499178
6,8750,9835.160766,9833.455000,9672.446526
7,21950,18936.634894,22048.630000,22058.082620
8,10500,12561.232829,12210.435000,12171.948259
9,8950,9477.663195,9549.410000,9391.507997
